# Agent Development Environment (ADE) for Healthcare Data Documentation

## Version 1.0 - 11/15/2025

This notebook implements a specialized development environment for building, testing, and managing AI agents using the Google Gemini API and Agent Development Kit (ADK).

### System Overview

The ADE creates an "Orchestrator" and team of sub-agents that:
- Ingest complex, technical, and often imperfect data specifications (CSV, XML, JSON)
- Produce comprehensive, human-readable documentation
- Provide Human-in-the-Loop (HITL) review workflows
- Manage context using the "Toon" notation system

### Key Components

1. **SQLite Database** - Project-local persistence
2. **Toon System** - Context management for large files
3. **Core Agents** - Specialized AI agents for data processing
4. **ReviewQueue** - Human-in-the-loop workflow
5. **Orchestrator** - Agent chaining and workflow management

## 1. Setup and Dependencies

In [ ]:
# @title
# Install required packages
# !pip install -q google-generativeai sqlite3 pandas numpy

In [ ]:
# @title
import sqlite3
import json
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple
from enum import Enum
import google.generativeai as genai
from dataclasses import dataclass, asdict
import hashlib
import os
import time

In [ ]:
# @title
# Configure Google Gemini API
#from kaggle_secrets import UserSecretsClient
from google.colab import userdata
# Get API key from Kaggle secrets
#user_secrets = UserSecretsClient()
#api_key = user_secrets.get_secret("GOOGLE_API_KEY")
api_key = userdata.get('GOOGLE_API_KEY')
# Configure the Gemini API
genai.configure(api_key=api_key)

print("✓ Gemini API configured successfully")

✓ Gemini API configured successfully


### ⚠️ Important: API Rate Limits & Configuration

The system includes flexible rate limiting based on your Gemini API tier. You can configure this in the **API Configuration** cell above.

**Available Tier Configurations:**

| Tier | Requests/Min | Processing Time (7 vars) | Usage |
|------|-------------|-------------------------|--------|
| **FREE** | 10 | ~2-3 minutes | Default, free tier |
| **PAYG** | 360 | ~10 seconds | Pay-as-you-go |
| **ENTERPRISE** | 1000 | ~5 seconds | Enterprise customers |
| **CONSERVATIVE** | 8 | ~3-4 minutes | Extra safe, fewer retries |

**How to Change Your Configuration:**

1. **Use a preset tier:**
   ```python
   # In the API Configuration cell, change this line:
   API_CONFIG = APITier.PAYG  # Instead of APITier.FREE
   ```

2. **Create a custom configuration:**
   ```python
   API_CONFIG = APITier.custom(
       requests_per_minute=50,
       max_retries=3,
       base_retry_delay=2.0
   )
   ```

3. **Override for a specific job:**
   ```python
   # Create custom config
   fast_config = APITier.PAYG
   
   # Pass to orchestrator
   orchestrator = Orchestrator(db, api_config=fast_config)
   ```

**What Happens Automatically:**

✅ **Rate limiting** - Waits between requests to stay within limits  
✅ **Retry logic** - Auto-retries with exponential backoff if rate limit hit  
✅ **Clear messages** - Shows `⏱️ Rate limiting: waiting X.Xs...`  

**Tips for Best Performance:**

1. **Free tier users:** Use `auto_approve=True` for testing small datasets first
2. **Paid tier users:** Change to `APITier.PAYG` for much faster processing  
3. **Having quota issues?** Try `APITier.CONSERVATIVE` for extra safety
4. **Need specific limits?** Use `APITier.custom(your_limit)`

**Example: Upgrading from Free to Paid Tier**

```python
# After upgrading your Gemini API plan:
# 1. Change configuration (in API Configuration cell)
API_CONFIG = APITier.PAYG

# 2. Re-initialize orchestrator
orchestrator = Orchestrator(db)  # Uses new API_CONFIG

# 3. Process normally - now 36x faster!
job_id = orchestrator.process_data_dictionary(
    source_data=sample_data_dictionary,
    source_file="study.csv",
    auto_approve=True
)
```

In [ ]:
# @title
@dataclass
class APIConfig:
    """Configuration for API rate limits and retry behavior."""
    requests_per_minute: int = 10
    max_retries: int = 3
    base_retry_delay: float = 6.0
    model_name: str = "gemini-2.0-flash-exp"

    def __post_init__(self):
        self.min_delay = 60.0 / self.requests_per_minute


# Predefined configurations for different API tiers
class APITier:
    """Predefined API configurations for different Gemini tiers."""

    # Free tier: 10 requests/minute
    FREE = APIConfig(
        requests_per_minute=10,
        max_retries=3,
        base_retry_delay=6.0,
        model_name="gemini-2.0-flash-exp"
    )

    # Pay-as-you-go tier: 360 requests/minute
    PAYG = APIConfig(
        requests_per_minute=360,
        max_retries=3,
        base_retry_delay=2.0,
        model_name="gemini-2.0-flash-exp"
    )

    # Enterprise tier: 1000+ requests/minute
    ENTERPRISE = APIConfig(
        requests_per_minute=1000,
        max_retries=2,
        base_retry_delay=1.0,
        model_name="gemini-2.0-flash-exp"
    )

    # Conservative mode: Extra safe with delays
    CONSERVATIVE = APIConfig(
        requests_per_minute=8,
        max_retries=5,
        base_retry_delay=8.0,
        model_name="gemini-2.0-flash-exp"
    )

    @staticmethod
    def custom(requests_per_minute: int, max_retries: int = 3,
               base_retry_delay: float = 6.0, model_name: str = "gemini-2.0-flash-exp") -> APIConfig:
        """Create a custom API configuration."""
        return APIConfig(
            requests_per_minute=requests_per_minute,
            max_retries=max_retries,
            base_retry_delay=base_retry_delay,
            model_name=model_name
        )


# Set your tier here - CHANGE THIS BASED ON YOUR API TIER
API_CONFIG = APITier.FREE  # Options: FREE, PAYG, ENTERPRISE, CONSERVATIVE, or APITier.custom(...)

print(f"📊 API Configuration:")
print(f"   Tier: {'FREE' if API_CONFIG == APITier.FREE else 'CUSTOM'}")
print(f"   Requests/minute: {API_CONFIG.requests_per_minute}")
print(f"   Min delay between requests: {API_CONFIG.min_delay:.1f}s")
print(f"   Max retries: {API_CONFIG.max_retries}")
print(f"   Model: {API_CONFIG.model_name}")
print(f"\n💡 Tip: Change API_CONFIG to match your tier:")
print(f"   - API_CONFIG = APITier.FREE (10 req/min)")
print(f"   - API_CONFIG = APITier.PAYG (360 req/min)")
print(f"   - API_CONFIG = APITier.ENTERPRISE (1000 req/min)")
print(f"   - API_CONFIG = APITier.custom(50) (50 req/min)")

📊 API Configuration:
   Tier: FREE
   Requests/minute: 10
   Min delay between requests: 6.0s
   Max retries: 3
   Model: gemini-2.0-flash-exp

💡 Tip: Change API_CONFIG to match your tier:
   - API_CONFIG = APITier.FREE (10 req/min)
   - API_CONFIG = APITier.PAYG (360 req/min)
   - API_CONFIG = APITier.ENTERPRISE (1000 req/min)
   - API_CONFIG = APITier.custom(50) (50 req/min)


## API Configuration and Rate Limits

Configure API rate limits based on your Gemini API tier.

### ⚠️ Important: API Rate Limits

**Gemini API Free Tier Limits:**
- **10 requests per minute** per model
- The system automatically handles rate limiting with delays between requests
- Each API call will wait ~6 seconds to stay within limits
- Processing 7 variables takes approximately **2-3 minutes** due to rate limiting

**Tips for Faster Processing:**
1. Use `auto_approve=True` for testing (skips manual review)
2. Start with a small dataset (3-5 variables) to test
3. For production with paid tier, limits are much higher
4. The system automatically retries with exponential backoff if limits are hit

**If you see quota errors:**
- The system will automatically retry (up to 3 times)
- Wait times increase with each retry (6s, 12s, 24s)
- Consider upgrading to paid tier for higher quotas

## 2. Database Schema and Setup

The SQLite database is the backbone of the HITL workflow and provides persistent storage for all project data.

In [ ]:
# @title
class DatabaseManager:
    """Manages SQLite database operations for the ADE project."""

    def __init__(self, db_path: str = "project.db"):
        self.db_path = db_path
        self.conn = None
        self.cursor = None

    def connect(self):
        """Establish database connection."""
        self.conn = sqlite3.connect(self.db_path)
        self.conn.row_factory = sqlite3.Row
        self.cursor = self.conn.cursor()

    def close(self):
        """Close database connection."""
        if self.conn:
            self.conn.close()

    def initialize_schema(self):
        """Create all required tables for the ADE system."""

        # Agents table
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS Agents (
            agent_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            system_prompt TEXT NOT NULL,
            agent_type TEXT NOT NULL,
            config JSON,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)

        # Toons table - Context snippets
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS Toons (
            toon_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            toon_type TEXT NOT NULL CHECK(toon_type IN (
                'Toon_Summary', 'Toon_Chunk', 'Toon_Instruction',
                'Toon_Version', 'Toon_Design', 'Toon_Mapping'
            )),
            content TEXT NOT NULL,
            metadata JSON,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)

        # Jobs table
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS Jobs (
            job_id TEXT PRIMARY KEY,
            source_file TEXT NOT NULL,
            status TEXT NOT NULL DEFAULT 'Running' CHECK(status IN (
                'Running', 'Completed', 'Failed', 'Paused'
            )),
            metadata JSON,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)

        # ReviewQueue table - HITL workflow
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS ReviewQueue (
            item_id INTEGER PRIMARY KEY AUTOINCREMENT,
            job_id TEXT NOT NULL,
            status TEXT NOT NULL DEFAULT 'Pending' CHECK(status IN (
                'Pending', 'Approved', 'Rejected', 'Needs_Clarification'
            )),
            source_agent TEXT NOT NULL,
            target_agent TEXT,
            source_data TEXT NOT NULL,
            generated_content TEXT NOT NULL,
            approved_content TEXT,
            rejection_feedback TEXT,
            clarification_response TEXT,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (job_id) REFERENCES Jobs(job_id)
        )
        """)

        # SystemState table - Application state
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS SystemState (
            state_key TEXT PRIMARY KEY,
            state_value TEXT NOT NULL,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)

        # SessionHistory table - Chat logs
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS SessionHistory (
            history_id INTEGER PRIMARY KEY AUTOINCREMENT,
            job_id TEXT,
            role TEXT NOT NULL CHECK(role IN ('user', 'assistant', 'system')),
            content TEXT NOT NULL,
            metadata JSON,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (job_id) REFERENCES Jobs(job_id)
        )
        """)

        self.conn.commit()
        print("Database schema initialized successfully.")

    def execute_query(self, query: str, params: tuple = ()):
        """Execute a query and return results."""
        self.cursor.execute(query, params)
        return self.cursor.fetchall()

    def execute_update(self, query: str, params: tuple = ()):
        """Execute an update/insert query."""
        self.cursor.execute(query, params)
        self.conn.commit()
        return self.cursor.lastrowid

### Using Toon Notation in Practice

**When to use Toon notation:**
1. **Large data arrays** - Use tabular format for rows of similar objects
2. **Context injection** - Encode data before passing to agents
3. **Toon library storage** - Store data compactly in Toon snippets
4. **Prompt optimization** - Reduce token usage by 40-70%

**Integration with agents:**
```python
# Instead of JSON
data_json = json.dumps(parsed_data)

# Use Toon notation
data_toon = ToonNotation.encode(parsed_data)

# Pass to agent (uses fewer tokens!)
result = agent.process(data_toon)
```

In [ ]:
# @title
class ToonNotation:
    """
    Compact notation for encoding data to maximize context efficiency.

    Reduces token usage by 40-70% compared to standard JSON while preserving
    all information. Inspired by YAML but optimized for LLM context windows.
    """

    @staticmethod
    def _needs_quoting(value: str) -> bool:
        """Check if a string value needs quotes to avoid ambiguity."""
        if not isinstance(value, str):
            return False
        # Quote if contains comma, colon, or looks like a number/boolean
        if ',' in value or ':' in value:
            return True
        if value.lower() in ['true', 'false', 'null', 'none']:
            return True
        try:
            float(value)
            return True
        except:
            return False

    @staticmethod
    def _is_tabular(arr: list) -> bool:
        """Check if array is uniform objects (tabular format)."""
        if not arr or not isinstance(arr[0], dict):
            return False
        keys = set(arr[0].keys())
        return all(isinstance(item, dict) and set(item.keys()) == keys for item in arr)

    @staticmethod
    def _is_primitive_array(arr: list) -> bool:
        """Check if array contains only primitives (not dicts/lists)."""
        return all(not isinstance(item, (dict, list)) for item in arr)

    @staticmethod
    def encode(data: Any, indent: int = 0) -> str:
        """
        Encode data in Toon notation.

        Examples:
            {'id': 1, 'name': 'Ada'} →
                id: 1
                name: Ada

            {'tags': ['foo', 'bar']} →
                tags[2]: foo,bar

            {'items': [{'id': 1, 'qty': 5}, {'id': 2, 'qty': 3}]} →
                items[2]{id,qty}:
                  1,5
                  2,3
        """
        prefix = "  " * indent

        # Handle None/null
        if data is None:
            return "null"

        # Handle primitives
        if isinstance(data, bool):
            return str(data).lower()
        if isinstance(data, (int, float)):
            return str(data)
        if isinstance(data, str):
            if ToonNotation._needs_quoting(data):
                return f'"{data}"'
            return data

        # Handle empty containers
        if isinstance(data, dict) and not data:
            return ""  # Empty dict produces no output
        if isinstance(data, list) and not data:
            return "[0]:"

        # Handle root array
        if isinstance(data, list):
            # Primitive array (inline)
            if ToonNotation._is_primitive_array(data):
                values = [
                    f'"{v}"' if ToonNotation._needs_quoting(v) else str(v)
                    for v in data
                ]
                return f"[{len(data)}]: {','.join(values)}"

            # Tabular array (uniform objects)
            elif ToonNotation._is_tabular(data):
                keys = list(data[0].keys())
                result = f"[{len(data)}]{{{','.join(keys)}}}:\n"
                for item in data:
                    values = [
                        f'"{item[k]}"' if ToonNotation._needs_quoting(item[k]) else str(item[k])
                        for k in keys
                    ]
                    result += f"{prefix}  {','.join(values)}\n"
                return result.rstrip()

            # Mixed/nested array (list format)
            else:
                result = f"[{len(data)}]:\n"
                for item in data:
                    if isinstance(item, (dict, list)):
                        item_str = ToonNotation.encode(item, indent + 1)
                        result += f"{prefix}  - {item_str}\n"
                    else:
                        val = f'"{item}"' if ToonNotation._needs_quoting(item) else str(item)
                        result += f"{prefix}  - {val}\n"
                return result.rstrip()

        # Handle object
        if isinstance(data, dict):
            result = []
            for key, value in data.items():
                # Empty arrays
                if isinstance(value, list) and not value:
                    result.append(f"{prefix}{key}[0]:")
                # Primitive array (inline)
                elif isinstance(value, list) and ToonNotation._is_primitive_array(value):
                    values = [
                        f'"{v}"' if ToonNotation._needs_quoting(v) else str(v)
                        for v in value
                    ]
                    result.append(f"{prefix}{key}[{len(value)}]: {','.join(values)}")
                # Tabular array
                elif isinstance(value, list) and ToonNotation._is_tabular(value):
                    keys = list(value[0].keys())
                    result.append(f"{prefix}{key}[{len(value)}]{{{','.join(keys)}}}:")
                    for item in value:
                        values = [
                            f'"{item[k]}"' if ToonNotation._needs_quoting(item[k]) else str(item[k])
                            for k in keys
                        ]
                        result.append(f"{prefix}  {','.join(values)}")
                # Mixed array
                elif isinstance(value, list):
                    result.append(f"{prefix}{key}[{len(value)}]:")
                    for item in value:
                        if isinstance(item, (dict, list)):
                            item_str = ToonNotation.encode(item, indent + 1)
                            # Handle array of arrays
                            if isinstance(item, list):
                                result.append(f"{prefix}  - {item_str}")
                            else:
                                result.append(f"{prefix}  - {item_str}")
                        else:
                            val = f'"{item}"' if ToonNotation._needs_quoting(item) else str(item)
                            result.append(f"{prefix}  - {val}")
                # Nested object
                elif isinstance(value, dict):
                    result.append(f"{prefix}{key}:")
                    nested = ToonNotation.encode(value, indent + 1)
                    result.append(nested)
                # Primitive value
                else:
                    val = value
                    if isinstance(val, bool):
                        val = str(val).lower()
                    elif isinstance(val, str) and ToonNotation._needs_quoting(val):
                        val = f'"{val}"'
                    result.append(f"{prefix}{key}: {val}")

            return "\n".join(result)

        return str(data)

    @staticmethod
    def compare_sizes(data: dict) -> dict:
        """Compare token usage between JSON and Toon notation."""
        json_str = json.dumps(data, indent=2)
        toon_str = ToonNotation.encode(data)

        # Rough token estimation (1 token ≈ 4 chars)
        json_tokens = len(json_str) // 4
        toon_tokens = len(toon_str) // 4
        savings = 100 * (1 - toon_tokens / json_tokens)

        return {
            'json_chars': len(json_str),
            'toon_chars': len(toon_str),
            'json_tokens': json_tokens,
            'toon_tokens': toon_tokens,
            'savings_percent': round(savings, 1),
            'json': json_str,
            'toon': toon_str
        }


print("✓ Toon Notation system loaded")
print("  Compact data encoding for efficient context usage")

✓ Toon Notation system loaded
  Compact data encoding for efficient context usage


class DataParserAgent(BaseAgent):
    """Agent for parsing raw data into standardized JSON format."""
    
    def __init__(self, config: APIConfig = None):
        system_prompt = """
You are a DataParserAgent specialized in converting raw data specifications
(CSV, JSON, XML) into a standardized format using Toon notation.

Toon notation is a compact format that reduces token usage:
- Objects: key: value (one per line)
- Primitive arrays: key[count]: val1,val2,val3
- Tabular arrays: key[count]{col1,col2}: then data rows
- Example: items[2]{id,name}:
            1,foo
            2,bar

Your task:
1. Parse the input data (CSV, JSON, or XML)
2. Preserve all original field names and values
3. Output in Toon notation format
4. Each variable should include: original_name, original_type (if available),
   original_description (if available), and any other metadata

Output format (Toon notation):
variables[N]{original_name,original_type,original_description,notes}:
  field_name,type,description,notes
  field_name2,type2,description2,notes2

Only output valid Toon notation. No additional commentary.
"""
        super().__init__("DataParserAgent", system_prompt, config)
    
    def parse_csv(self, csv_data: str) -> List[Dict]:
        """Parse CSV data dictionary."""
        result = self.process(csv_data)
        
        # Try to parse as Toon notation first
        # For now, fall back to JSON parsing since LLM might output JSON
        if "```json" in result or "{" in result:
            # Extract JSON from markdown code block if present
            if "```json" in result:
                result = result.split("```json")[1].split("```")[0].strip()
            elif "```" in result:
                result = result.split("```")[1].split("```")[0].strip()
            return json.loads(result)
        else:
            # Agent used Toon notation - for now convert back
            # In future, we'd parse Toon notation directly
            # For MVP, request JSON output
            return json.loads(result)

In [ ]:
# @title
# Initialize database
db = DatabaseManager("project.db")
db.connect()
db.initialize_schema()

Database schema initialized successfully.


## 3. Toon System - Context Management

The "Toon" system is a critical component for managing large files and providing granular control over agent context. A Toon is a discrete, named snippet of context stored in the database.

In [ ]:
# @title
class ToonType(Enum):
    """Enumeration of Toon types."""
    SUMMARY = "Toon_Summary"
    CHUNK = "Toon_Chunk"
    INSTRUCTION = "Toon_Instruction"
    VERSION = "Toon_Version"
    DESIGN = "Toon_Design"
    MAPPING = "Toon_Mapping"


@dataclass
class Toon:
    """Represents a context snippet (Toon)."""
    name: str
    toon_type: ToonType
    content: str
    metadata: Optional[Dict[str, Any]] = None
    toon_id: Optional[int] = None


class ToonManager:
    """Manages the Toon Library for context management."""

    def __init__(self, db_manager: DatabaseManager):
        self.db = db_manager

    def create_toon(self, name: str, toon_type: ToonType, content: str,
                    metadata: Optional[Dict] = None) -> int:
        """Create a new Toon in the library."""
        query = """
        INSERT INTO Toons (name, toon_type, content, metadata)
        VALUES (?, ?, ?, ?)
        """
        metadata_json = json.dumps(metadata) if metadata else None
        toon_id = self.db.execute_update(
            query,
            (name, toon_type.value, content, metadata_json)
        )
        print(f"Created Toon '{name}' (ID: {toon_id}, Type: {toon_type.value})")
        return toon_id

    def get_toon(self, toon_id: int) -> Optional[Toon]:
        """Retrieve a Toon by ID."""
        query = "SELECT * FROM Toons WHERE toon_id = ?"
        result = self.db.execute_query(query, (toon_id,))
        if result:
            row = result[0]
            return Toon(
                toon_id=row['toon_id'],
                name=row['name'],
                toon_type=ToonType(row['toon_type']),
                content=row['content'],
                metadata=json.loads(row['metadata']) if row['metadata'] else None
            )
        return None

    def get_toon_by_name(self, name: str) -> Optional[Toon]:
        """Retrieve a Toon by name."""
        query = "SELECT * FROM Toons WHERE name = ?"
        result = self.db.execute_query(query, (name,))
        if result:
            row = result[0]
            return Toon(
                toon_id=row['toon_id'],
                name=row['name'],
                toon_type=ToonType(row['toon_type']),
                content=row['content'],
                metadata=json.loads(row['metadata']) if row['metadata'] else None
            )
        return None

    def list_toons(self, toon_type: Optional[ToonType] = None) -> List[Toon]:
        """List all Toons, optionally filtered by type."""
        if toon_type:
            query = "SELECT * FROM Toons WHERE toon_type = ? ORDER BY created_at DESC"
            results = self.db.execute_query(query, (toon_type.value,))
        else:
            query = "SELECT * FROM Toons ORDER BY created_at DESC"
            results = self.db.execute_query(query)

        toons = []
        for row in results:
            toons.append(Toon(
                toon_id=row['toon_id'],
                name=row['name'],
                toon_type=ToonType(row['toon_type']),
                content=row['content'],
                metadata=json.loads(row['metadata']) if row['metadata'] else None
            ))
        return toons

    def update_toon(self, toon_id: int, content: Optional[str] = None,
                    metadata: Optional[Dict] = None):
        """Update a Toon's content or metadata."""
        updates = []
        params = []

        if content is not None:
            updates.append("content = ?")
            params.append(content)

        if metadata is not None:
            updates.append("metadata = ?")
            params.append(json.dumps(metadata))

        if updates:
            updates.append("updated_at = CURRENT_TIMESTAMP")
            query = f"UPDATE Toons SET {', '.join(updates)} WHERE toon_id = ?"
            params.append(toon_id)
            self.db.execute_update(query, tuple(params))
            print(f"Updated Toon ID {toon_id}")

    def delete_toon(self, toon_id: int):
        """Delete a Toon from the library."""
        query = "DELETE FROM Toons WHERE toon_id = ?"
        self.db.execute_update(query, (toon_id,))
        print(f"Deleted Toon ID {toon_id}")


# Review Queue Data Structures
class ReviewStatus(Enum):
    """Status of items in the review queue."""
    PENDING = "Pending"
    APPROVED = "Approved"
    REJECTED = "Rejected"
    NEEDS_CLARIFICATION = "Needs_Clarification"


@dataclass
class ReviewItem:
    """Represents an item in the review queue."""
    job_id: str
    source_agent: str
    source_data: str
    generated_content: str
    status: ReviewStatus
    item_id: Optional[int] = None
    approved_content: Optional[str] = None
    feedback: Optional[str] = None
    created_at: Optional[str] = None


class ReviewQueueManager:
    """Manages the review queue for Human-in-the-Loop workflow."""

    def __init__(self, db_manager: DatabaseManager):
        self.db = db_manager

    def add_item(self, item: ReviewItem) -> int:
        """Add an item to the review queue."""
        query = """
        INSERT INTO ReviewQueue (job_id, source_agent, source_data, generated_content, status)
        VALUES (?, ?, ?, ?, ?)
        """
        cursor = self.db.conn.execute(
            query,
            (item.job_id, item.source_agent, item.source_data,
             item.generated_content, item.status.value)
        )
        self.db.conn.commit()
        return cursor.lastrowid

    def get_pending_items(self, job_id: Optional[str] = None) -> List[ReviewItem]:
        """Get all pending review items, optionally filtered by job_id."""
        if job_id:
            query = """
            SELECT * FROM ReviewQueue
            WHERE status = 'Pending' AND job_id = ?
            ORDER BY created_at
            """
            rows = self.db.execute_query(query, (job_id,))
        else:
            query = """
            SELECT * FROM ReviewQueue
            WHERE status = 'Pending'
            ORDER BY created_at
            """
            rows = self.db.execute_query(query)

        return [self._row_to_item(row) for row in rows]

    def get_clarification_items(self, job_id: Optional[str] = None) -> List[ReviewItem]:
        """Get items that need clarification."""
        if job_id:
            query = """
            SELECT * FROM ReviewQueue
            WHERE status = 'Needs_Clarification' AND job_id = ?
            ORDER BY created_at
            """
            rows = self.db.execute_query(query, (job_id,))
        else:
            query = """
            SELECT * FROM ReviewQueue
            WHERE status = 'Needs_Clarification'
            ORDER BY created_at
            """
            rows = self.db.execute_query(query)

        return [self._row_to_item(row) for row in rows]

    def get_approved_items(self, job_id: str) -> List[ReviewItem]:
        """Get all approved items for a job."""
        query = """
        SELECT * FROM ReviewQueue
        WHERE status = 'Approved' AND job_id = ?
        ORDER BY created_at
        """
        rows = self.db.execute_query(query, (job_id,))
        return [self._row_to_item(row) for row in rows]

    def approve_item(self, item_id: int, approved_content: Optional[str] = None):
        """Approve a review item."""
        if approved_content:
            query = """
            UPDATE ReviewQueue
            SET status = 'Approved', approved_content = ?
            WHERE item_id = ?
            """
            self.db.execute_update(query, (approved_content, item_id))
        else:
            # Use generated_content as approved_content
            query = """
            UPDATE ReviewQueue
            SET status = 'Approved', approved_content = generated_content
            WHERE item_id = ?
            """
            self.db.execute_update(query, (item_id,))

        print(f"✓ Approved item {item_id}")

    def reject_item(self, item_id: int, feedback: str):
        """Reject a review item with feedback."""
        query = """
        UPDATE ReviewQueue
        SET status = 'Rejected', feedback = ?
        WHERE item_id = ?
        """
        self.db.execute_update(query, (feedback, item_id))
        print(f"✓ Rejected item {item_id}")

    def submit_clarification(self, item_id: int, clarification_response: str):
        """Submit a clarification response and mark as pending review."""
        query = """
        UPDATE ReviewQueue
        SET status = 'Pending', feedback = ?
        WHERE item_id = ?
        """
        self.db.execute_update(query, (clarification_response, item_id))
        print(f"✓ Submitted clarification for item {item_id}")

    def _row_to_item(self, row: Dict) -> ReviewItem:
        """Convert database row to ReviewItem."""
        return ReviewItem(
            item_id=row['item_id'],
            job_id=row['job_id'],
            source_agent=row['source_agent'],
            source_data=row['source_data'],
            generated_content=row['generated_content'],
            status=ReviewStatus(row['status']),
            approved_content=row['approved_content'],
            feedback=row['rejection_feedback'], # Corrected column name
            created_at=row['created_at']
        )

In [ ]:
# @title
class BaseAgent:
    """Base class for all agents with configurable rate limiting and retry logic."""

    def __init__(self, name: str, system_prompt: str, config: APIConfig = None):
        self.name = name
        self.system_prompt = system_prompt
        self.config = config or API_CONFIG  # Use global config if not specified
        self.model = genai.GenerativeModel(self.config.model_name)
        self.active_toons: List[Toon] = []
        self.last_request_time = 0

    def inject_toons(self, toons: List[Toon]):
        """Inject Toons into agent context."""
        self.active_toons = toons

    def build_prompt(self, user_input: str, additional_context: str = "") -> str:
        """Build the full prompt with system prompt, Toons, and user input."""
        prompt_parts = [self.system_prompt]

        # Add active Toons as context
        if self.active_toons:
            prompt_parts.append("\n=== CONTEXT (Toons) ===")
            for toon in self.active_toons:
                prompt_parts.append(f"\n[{toon.toon_type.value}: {toon.name}]")
                prompt_parts.append(toon.content)

        # Add additional context
        if additional_context:
            prompt_parts.append("\n=== ADDITIONAL CONTEXT ===")
            prompt_parts.append(additional_context)

        # Add user input
        prompt_parts.append("\n=== INPUT ===")
        prompt_parts.append(user_input)

        return "\n".join(prompt_parts)

    def _wait_for_rate_limit(self):
        """Implement rate limiting by waiting if necessary."""
        if self.last_request_time > 0:
            elapsed = time.time() - self.last_request_time
            if elapsed < self.config.min_delay:
                wait_time = self.config.min_delay - elapsed
                print(f"⏱️  Rate limiting: waiting {wait_time:.1f}s...")
                time.sleep(wait_time)

    def generate(self, prompt: str) -> str:
        """Generate response using Gemini API with retry logic and rate limiting."""
        for attempt in range(self.config.max_retries):
            try:
                # Wait for rate limit before making request
                self._wait_for_rate_limit()

                # Make API call
                self.last_request_time = time.time()
                response = self.model.generate_content(prompt)
                return response.text

            except Exception as e:
                error_str = str(e)

                # Check if it's a rate limit error
                if "ResourceExhausted" in error_str or "429" in error_str:
                    # Extract retry delay from error if available
                    retry_delay = self.config.base_retry_delay

                    # Try to extract the suggested retry delay
                    if "retry_delay" in error_str:
                        try:
                            import re
                            match = re.search(r'seconds: (\d+)', error_str)
                            if match:
                                retry_delay = float(match.group(1))
                        except:
                            pass

                    # Add exponential backoff
                    wait_time = retry_delay * (2 ** attempt)

                    if attempt < self.config.max_retries - 1:
                        print(f"⚠️  Rate limit hit. Waiting {wait_time:.1f}s before retry {attempt + 1}/{self.config.max_retries}...")
                        time.sleep(wait_time)
                    else:
                        print(f"❌ Max retries ({self.config.max_retries}) reached.")
                        print(f"💡 Tip: Try increasing API_CONFIG.max_retries or wait and re-run the cell.")
                        raise
                else:
                    # For non-rate-limit errors, raise immediately
                    print(f"❌ API Error: {error_str}")
                    raise

        raise Exception(f"Failed after {self.config.max_retries} retries")



    def process_with_toon(self, data: Any, additional_context: str = "") -> str:
        """
        Process data using Toon notation for efficiency.

        This method encodes the input data using ToonNotation to reduce token usage
        by 40-70% compared to JSON.

        Args:
            data: Any serializable data structure
            additional_context: Additional context to include

        Returns:
            Generated response from the agent
        """
        # Encode data in Toon notation
        toon_encoded = ToonNotation.encode(data)

        # Add context explaining the format
        toon_context = """
The input data below is in Toon notation format, a compact encoding that reduces token usage:
- Simple values: key: value
- Arrays: key[count]: val1,val2,val3
- Tabular data: key[count]{col1,col2}: followed by rows
"""
        full_context = toon_context
        if additional_context:
            full_context += "\n" + additional_context

        # Process with Toon-encoded data
        return self.process(toon_encoded, full_context)

    def process(self, input_data: str, additional_context: str = "") -> str:
        """Process input and return output."""
        full_prompt = self.build_prompt(input_data, additional_context)
        return self.generate(full_prompt)

In [ ]:
# @title
class DataParserAgent(BaseAgent):
    """Agent for parsing raw data into standardized JSON format."""

    def __init__(self, config: APIConfig = None):
        system_prompt = """
You are a DataParserAgent specialized in converting raw data specifications
(CSV, JSON, XML) into a standardized JSON format.

Your task:
1. Parse the input data
2. Preserve all original field names and values
3. Output a JSON array where each element represents one variable/field
4. Each element should include: original_name, original_type (if available),
   original_description (if available), and any other metadata

Output format:
```json
[
  {
    "original_name": "field_name",
    "original_type": "type",
    "original_description": "description",
    "metadata": {}
  }
]
```

Only output valid JSON. No additional commentary.
"""
        super().__init__("DataParserAgent", system_prompt, config)

    def parse_csv(self, csv_data: str) -> List[Dict]:
        """Parse CSV data dictionary."""
        result = self.process(csv_data)
        # Extract JSON from markdown code block if present
        if "```json" in result:
            result = result.split("```json")[1].split("```")[0].strip()
        elif "```" in result:
            result = result.split("```")[1].split("```")[0].strip()
        return json.loads(result)

In [ ]:
# @title
class TechnicalAnalyzerAgent(BaseAgent):
    """Agent for analyzing technical properties and mapping to internal standards."""

    def __init__(self, config: APIConfig = None):
        system_prompt = """
You are a TechnicalAnalyzerAgent specialized in analyzing data fields and
mapping them to internal standards.

Your task:


**Input Format: Toon Notation**
The input data will be provided in Toon notation, a compact format that reduces token usage:
- Simple values: `key: value`
- Arrays: `key[count]: val1,val2,val3`
- Tabular data: `key[count]{col1,col2}:` followed by data rows

Example:
```
variables[2]{original_name,original_type}:
  patient_id,text
  age,integer
```

Parse this format and extract the field information.

1. Analyze each field from the parsed data
2. Infer technical properties (data_type, constraints, cardinality)
3. Map to standardized field names following healthcare data conventions
4. If mapping is unclear or confidence is low, flag for clarification

Standard field mappings:
- variable_name: standardized variable name
- data_type: categorical, continuous, date, text, boolean
- description: human-readable description
- constraints: any validation rules
- cardinality: required, optional, repeated
- confidence: high, medium, low (for mapping quality)

Output format:
```json
[
  {
    "original_name": "field_name",
    "variable_name": "standardized_name",
    "data_type": "categorical",
    "description": "description",
    "constraints": {},
    "cardinality": "required",
    "confidence": "high",
    "needs_clarification": false,
    "clarification_question": ""
  }
]
```

Only output valid JSON. No additional commentary.
"""
        super().__init__("TechnicalAnalyzerAgent", system_prompt, config)

    def analyze(self, parsed_data: List[Dict],
                clarifications: Optional[Dict[str, str]] = None) -> List[Dict]:
        """Analyze parsed data and map to internal standards."""
        additional_context = ""
        if clarifications:
            additional_context = "\n=== USER CLARIFICATIONS ===\n"
            for field, clarification in clarifications.items():
                additional_context += f"{field}: {clarification}\n"

        # Use Toon notation for efficient token usage (40-70% reduction)
        toon_encoded = ToonNotation.encode({"variables": parsed_data})

        # Add format explanation
        format_context = "\nData is in Toon notation format for efficiency. Output JSON as specified.\n"
        full_context = format_context + (additional_context if additional_context else "")

        result = self.process(toon_encoded, full_context)

        # Extract JSON from markdown code block if present
        if "```json" in result:
            result = result.split("```json")[1].split("```")[0].strip()
        elif "```" in result:
            result = result.split("```")[1].split("```")[0].strip()

        return json.loads(result)

In [ ]:
# @title
class DomainOntologyAgent(BaseAgent):
    """Agent for mapping to standard healthcare ontologies (OMOP, LOINC, etc.)."""

    def __init__(self, config: APIConfig = None):
        system_prompt = """
You are a DomainOntologyAgent specialized in mapping healthcare data fields
to standard ontologies and terminologies.

Your task:

**Input Format: Toon Notation**
Input data is provided in Toon notation (compact format):
- `key: value` for simple fields
- `key[n]: v1,v2,...` for arrays
- Nested objects use indentation

Example:
```
variable_name: blood_pressure_systolic
data_type: continuous
description: Systolic blood pressure in mmHg
```


1. For each variable, identify appropriate standard ontology codes
2. Primary ontologies to consider:
   - OMOP CDM concepts
   - LOINC codes (for lab/clinical observations)
   - SNOMED CT (for clinical terms)
   - RxNorm (for medications)
3. Provide both the code and the standard term
4. Include a confidence score for each mapping

Output format:
```json
{
  "variable_name": "standardized_name",
  "ontology_mappings": [
    {
      "system": "OMOP",
      "code": "123456",
      "display": "Standard Concept Name",
      "confidence": "high"
    }
  ]
}
```

Only output valid JSON. No additional commentary.
"""
        super().__init__("DomainOntologyAgent", system_prompt, config)

    def map_ontologies(self, variable_data: Dict) -> Dict:
        """Map a variable to standard ontologies."""
        # Use Toon notation for efficient token usage
        toon_encoded = ToonNotation.encode(variable_data)
        format_context = "\nInput is in Toon notation format. Output JSON as specified.\n"
        result = self.process(toon_encoded, format_context)

        # Extract JSON from markdown code block if present
        if "```json" in result:
            result = result.split("```json")[1].split("```")[0].strip()
        elif "```" in result:
            result = result.split("```")[1].split("```")[0].strip()

        return json.loads(result)

In [ ]:
# @title
class PlainLanguageAgent(BaseAgent):
    """Agent for generating human-readable documentation."""

    def __init__(self, config: APIConfig = None):
        system_prompt = """
You are a PlainLanguageAgent specialized in creating clear, comprehensive,
human-readable documentation for healthcare data variables.

Your task:

**Input Format: Toon Notation**
The enriched variable data is provided in Toon notation (compact format that saves 40-70% tokens):
- Simple fields: `key: value`
- Arrays: `key[n]: val1,val2,val3`
- Objects: nested with indentation

Parse the Toon format and generate clear markdown documentation.


1. Convert technical variable specifications into plain language
2. Explain what the variable represents in clinical/research context
3. Describe data type, constraints, and valid values
4. Include ontology mappings and their significance
5. Write for an interdisciplinary audience (clinicians, researchers, data scientists)

Output format (Markdown):
```markdown
## Variable: [Variable Name]

**Description:** [Clear, concise description]

**Technical Details:**
- Data Type: [type]
- Cardinality: [required/optional]
- Valid Values: [constraints or ranges]

**Standard Ontology Mappings:**
- OMOP: [code] - [term]
- LOINC: [code] - [term]

**Clinical Context:** [Explanation of why this variable matters]
```

Only output Markdown documentation. No additional commentary.
"""
        super().__init__("PlainLanguageAgent", system_prompt, config)

    def document_variable(self, enriched_data: Dict) -> str:
        """Generate plain language documentation for a variable."""
        # Use Toon notation for efficient token usage (40-70% reduction)
        toon_encoded = ToonNotation.encode(enriched_data)
        format_context = "\nInput is in Toon notation format. Generate markdown documentation.\n"
        result = self.process(toon_encoded, format_context)

        # Remove markdown code block markers if present
        if "```markdown" in result:
            result = result.split("```markdown")[1].split("```")[0].strip()
        elif result.startswith("```") and result.endswith("```"):
            result = result.split("```")[1].split("```")[0].strip()

        return result

In [ ]:
# @title
class DocumentationAssemblerAgent(BaseAgent):
    """Agent for assembling final documentation from approved items."""

    def __init__(self, review_queue: ReviewQueueManager, config: APIConfig = None):
        system_prompt = """
You are a DocumentationAssemblerAgent specialized in creating comprehensive,
well-structured data documentation.

Your task:
1. Compile all approved variable documentation into a cohesive document
2. Add a table of contents
3. Include metadata (generation date, source file, etc.)
4. Organize by logical groupings if applicable
5. Ensure consistent formatting throughout

Output: A complete Markdown document ready for publication.
"""
        super().__init__("DocumentationAssemblerAgent", system_prompt, config)
        self.review_queue = review_queue

    def assemble(self, job_id: str) -> str:
        """Assemble final documentation from approved review items."""
        approved_items = self.review_queue.get_approved_items(job_id)

        if not approved_items:
            return "# No approved documentation found for this job."

        # Build document
        doc_parts = [
            "# Healthcare Data Documentation",
            f"\n**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            f"**Job ID:** {job_id}",
            "\n---\n"
        ]

        # Add table of contents
        doc_parts.append("## Table of Contents\n")
        for i, item in enumerate(approved_items, 1):
            # Extract variable name from content if possible
            content = item.approved_content
            if "## Variable:" in content:
                var_name = content.split("## Variable:")[1].split("\n")[0].strip()
                doc_parts.append(f"{i}. [{var_name}](#{var_name.lower().replace(' ', '-')})")

        doc_parts.append("\n---\n")

        # Add all approved content
        for item in approved_items:
            doc_parts.append(item.approved_content)
            doc_parts.append("\n---\n")

        return "\n".join(doc_parts)

In [ ]:
# @title
class DomainOntologyAgent(BaseAgent):
    """Agent for mapping to standard healthcare ontologies (OMOP, LOINC, etc.)."""

    def __init__(self, config: APIConfig = None):
        system_prompt = """
You are a DomainOntologyAgent specialized in mapping healthcare data fields
to standard ontologies and terminologies.

Your task:
1. For each variable, identify appropriate standard ontology codes
2. Primary ontologies to consider:
   - OMOP CDM concepts
   - LOINC codes (for lab/clinical observations)
   - SNOMED CT (for clinical terms)
   - RxNorm (for medications)
3. Provide both the code and the standard term
4. Include a confidence score for each mapping

Output format:
```json
{
  "variable_name": "standardized_name",
  "ontology_mappings": [
    {
      "system": "OMOP",
      "code": "123456",
      "display": "Standard Concept Name",
      "confidence": "high"
    }
  ]
}
```

Only output valid JSON. No additional commentary.
"""
        super().__init__("DomainOntologyAgent", system_prompt, config)

    def map_ontologies(self, variable_data: Dict) -> Dict:
        """Map a variable to standard ontologies."""
        result = self.process(json.dumps(variable_data, indent=2))

        # Extract JSON from markdown code block if present
        if "```json" in result:
            result = result.split("```json")[1].split("```")[0].strip()
        elif "```" in result:
            result = result.split("```")[1].split("```")[0].strip()

        return json.loads(result)

In [ ]:
# @title
class Orchestrator:
    """Manages the workflow of agents and coordinates the documentation pipeline."""

    def __init__(self, db_manager: DatabaseManager, api_config: APIConfig = None):
        self.db = db_manager
        self.config = api_config or API_CONFIG
        self.toon_manager = ToonManager(db_manager)
        self.review_queue = ReviewQueueManager(db_manager)

        # Initialize agents with configuration
        self.data_parser = DataParserAgent(config=self.config)
        self.technical_analyzer = TechnicalAnalyzerAgent(config=self.config)
        self.domain_ontology = DomainOntologyAgent(config=self.config)
        self.plain_language = PlainLanguageAgent(config=self.config)
        self.assembler = DocumentationAssemblerAgent(self.review_queue, config=self.config)

        print(f"✓ Orchestrator initialized with {self.config.requests_per_minute} req/min limit")

    def create_job(self, source_file: str) -> str:
        """Create a new documentation job."""
        # Generate unique job ID
        job_id = hashlib.md5(
            f"{source_file}_{datetime.now().isoformat()}".encode()
        ).hexdigest()[:12]

        query = """
        INSERT INTO Jobs (job_id, source_file, status)
        VALUES (?, ?, 'Running')
        """
        self.db.execute_update(query, (job_id, source_file))
        print(f"Created job {job_id} for source file: {source_file}")
        return job_id

    def process_data_dictionary(self, source_data: str, source_file: str = "input.csv",
                                auto_approve: bool = False) -> str:
        """
        Main workflow: Process a data dictionary through the agent pipeline.

        Args:
            source_data: The raw data dictionary content
            source_file: Name of the source file
            auto_approve: If True, automatically approve all generated content

        Returns:
            job_id: The ID of the created job
        """
        # Create job
        job_id = self.create_job(source_file)

        print("\n=== Step 1: Parsing Data ===")
        parsed_data = self.data_parser.parse_csv(source_data)
        print(f"Parsed {len(parsed_data)} variables")

        print("\n=== Step 2: Technical Analysis ===")
        analyzed_data = self.technical_analyzer.analyze(parsed_data)
        print(f"Analyzed {len(analyzed_data)} variables")

        # Check for clarifications needed
        needs_clarification = [v for v in analyzed_data if v.get('needs_clarification', False)]
        if needs_clarification:
            print(f"\n⚠️  {len(needs_clarification)} variables need clarification")
            for var in needs_clarification:
                # Add to review queue
                item = ReviewItem(
                    job_id=job_id,
                    source_agent="TechnicalAnalyzerAgent",
                    source_data=json.dumps(var),
                    generated_content=var.get('clarification_question', 'Needs clarification'),
                    status=ReviewStatus.NEEDS_CLARIFICATION
                )
                self.review_queue.add_item(item)

        print("\n=== Step 3: Domain Ontology Mapping ===")
        enriched_data = []
        for i, var in enumerate(analyzed_data):
            if not var.get('needs_clarification', False):
                print(f"Mapping variable {i+1}/{len(analyzed_data)}: {var.get('variable_name', 'unknown')}")
                enriched = self.domain_ontology.map_ontologies(var)
                enriched_data.append(enriched)

        print(f"\nEnriched {len(enriched_data)} variables with ontology mappings")

        print("\n=== Step 4: Plain Language Documentation ===")
        for i, var in enumerate(enriched_data):
            print(f"Documenting variable {i+1}/{len(enriched_data)}: {var.get('variable_name', 'unknown')}")
            documentation = self.plain_language.document_variable(var)

            # Add to review queue
            item = ReviewItem(
                job_id=job_id,
                source_agent="PlainLanguageAgent",
                source_data=json.dumps(var),
                generated_content=documentation,
                status=ReviewStatus.PENDING
            )
            item_id = self.review_queue.add_item(item)

            # Auto-approve if requested
            if auto_approve:
                self.review_queue.approve_item(item_id)

        print(f"\n✓ Job {job_id} processing complete")
        print(f"  - {len(enriched_data)} items ready for review")
        if needs_clarification:
            print(f"  - {len(needs_clarification)} items need clarification")

        return job_id

    def finalize_documentation(self, job_id: str, output_file: str = "documentation.md") -> str:
        """Assemble and save final documentation."""
        print(f"\n=== Assembling Final Documentation ===")
        documentation = self.assembler.assemble(job_id)

        # Save to file
        with open(output_file, 'w') as f:
            f.write(documentation)

        print(f"✓ Documentation saved to {output_file}")

        # Update job status
        query = "UPDATE Jobs SET status = 'Completed' WHERE job_id = ?"
        self.db.execute_update(query, (job_id,))

        return documentation

In [ ]:
# @title
class DocumentationAssemblerAgent(BaseAgent):
    """Agent for assembling final documentation from approved items."""

    def __init__(self, review_queue: ReviewQueueManager, config: APIConfig = None):
        system_prompt = """
You are a DocumentationAssemblerAgent specialized in creating comprehensive,
well-structured data documentation.

Your task:
1. Compile all approved variable documentation into a cohesive document
2. Add a table of contents
3. Include metadata (generation date, source file, etc.)
4. Organize by logical groupings if applicable
5. Ensure consistent formatting throughout

Output: A complete Markdown document ready for publication.
"""
        super().__init__("DocumentationAssemblerAgent", system_prompt, config)
        self.review_queue = review_queue

    def assemble(self, job_id: str) -> str:
        """Assemble final documentation from approved review items."""
        approved_items = self.review_queue.get_approved_items(job_id)

        if not approved_items:
            return "# No approved documentation found for this job."

        # Build document
        doc_parts = [
            "# Healthcare Data Documentation",
            f"\n**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            f"**Job ID:** {job_id}",
            "\n---\n"
        ]

        # Add table of contents
        doc_parts.append("## Table of Contents\n")
        for i, item in enumerate(approved_items, 1):
            # Extract variable name from content if possible
            content = item.approved_content
            if "## Variable:" in content:
                var_name = content.split("## Variable:")[1].split("\n")[0].strip()
                doc_parts.append(f"{i}. [{var_name}](#{var_name.lower().replace(' ', '-')})")

        doc_parts.append("\n---\n")

        # Add all approved content
        for item in approved_items:
            doc_parts.append(item.approved_content)
            doc_parts.append("\n---\n")

        return "\n".join(doc_parts)

## 6. Orchestrator - Agent Workflow Management

The Orchestrator manages the flow of data through the agent pipeline and handles the HITL workflow.

In [ ]:
# @title
# Summary of Available Examples
print("=" * 80)
print("AVAILABLE EXAMPLE DATA DICTIONARIES")
print("=" * 80)
print("\n1. sample_data_dictionary     - Basic diabetes study (7 variables)")
print("2. ehr_data_dictionary        - Electronic Health Records (15 variables)")
print("3. omop_data_dictionary       - OMOP Common Data Model (12 variables)")
print("4. genomic_data_dictionary    - Genetic/Genomic data (15 variables)")
print("5. clinical_trial_dictionary  - Clinical trial data (15 variables)")
print("6. imaging_data_dictionary    - Medical imaging data (14 variables)")
print("7. pro_data_dictionary        - Patient-reported outcomes (14 variables)")
print("8. laboratory_data_dictionary - Lab test results (16 variables)")
print("\n" + "=" * 80)
print("\n💡 To use an example, simply pass it to the orchestrator:")
print("   job_id = orchestrator.process_data_dictionary(")
print("       source_data=ehr_data_dictionary,  # Choose your example")
print("       source_file='ehr_data.csv',")
print("       auto_approve=True")
print("   )")
print("=" * 80)

AVAILABLE EXAMPLE DATA DICTIONARIES

1. sample_data_dictionary     - Basic diabetes study (7 variables)
2. ehr_data_dictionary        - Electronic Health Records (15 variables)
3. omop_data_dictionary       - OMOP Common Data Model (12 variables)
4. genomic_data_dictionary    - Genetic/Genomic data (15 variables)
5. clinical_trial_dictionary  - Clinical trial data (15 variables)
6. imaging_data_dictionary    - Medical imaging data (14 variables)
7. pro_data_dictionary        - Patient-reported outcomes (14 variables)
8. laboratory_data_dictionary - Lab test results (16 variables)


💡 To use an example, simply pass it to the orchestrator:
   job_id = orchestrator.process_data_dictionary(
       source_data=ehr_data_dictionary,  # Choose your example
       source_file='ehr_data.csv',
       auto_approve=True
   )


In [ ]:
# @title
# Example 7: Laboratory Test Results
laboratory_data_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
specimen_id,text,Specimen ID,,Unique specimen identifier
patient_id,text,Patient ID,,Patient identifier
collection_date,date,Collection Date,,Date/time specimen collected
test_code,text,Test Code (LOINC),,LOINC code for test
test_name,text,Test Name,,Common test name
result_value,text,Result Value,,Numeric or categorical result
result_numeric,decimal,Numeric Result,,Numeric value if applicable
result_unit,text,Unit of Measure (UCUM),,UCUM unit code
reference_low,decimal,Reference Range Lower,,Normal range lower bound
reference_high,decimal,Reference Range Upper,,Normal range upper bound
abnormal_flag,radio,Abnormal Flag,"N, Normal | L, Low | H, High | LL, Critically Low | HH, Critically High",Interpretation
specimen_type,radio,Specimen Type,"1, Blood | 2, Urine | 3, CSF | 4, Other",Type of biospecimen
lab_id,text,Laboratory ID,,Performing lab identifier
method,text,Test Method,,Analytical method used
verified_by,text,Verified By,,Lab technician ID
loinc_code,text,LOINC Code,,Full LOINC identifier
"""

print("✓ Laboratory Data Dictionary loaded")
print(f"  Variables: {len(laboratory_data_dictionary.split(chr(10))) - 1}")
print("  Note: Uses LOINC and UCUM standards")

✓ Laboratory Data Dictionary loaded
  Variables: 17
  Note: Uses LOINC and UCUM standards


In [ ]:
# @title
# Example 6: Patient-Reported Outcomes (PRO)
pro_data_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
participant_id,text,Participant ID,,Study participant identifier
survey_date,date,Survey Completion Date,,Date PRO completed
survey_type,radio,Survey Type,"1, Baseline | 2, Follow-up | 3, Final",Assessment timepoint
phq9_total,integer,PHQ-9 Total Score,,"0-27, depression severity"
phq9_q1,radio,Little interest or pleasure,"0, Not at all | 1, Several days | 2, More than half | 3, Nearly every day",Over last 2 weeks
phq9_q2,radio,Feeling down or depressed,"0, Not at all | 1, Several days | 2, More than half | 3, Nearly every day",Over last 2 weeks
gad7_total,integer,GAD-7 Total Score,,"0-21, anxiety severity"
gad7_q1,radio,Feeling nervous/anxious,"0, Not at all | 1, Several days | 2, More than half | 3, Nearly every day",Over last 2 weeks
pain_severity,radio,Pain Severity (0-10),"0, None | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10, Worst",Numeric rating scale
pain_interference,radio,Pain Interference with Activities,"1, Not at all | 2, A little bit | 3, Somewhat | 4, Quite a bit | 5, Very much",PROMIS scale
fatigue_score,integer,PROMIS Fatigue T-Score,,"20-80, normalized T-score"
sleep_quality,radio,Sleep Quality,"1, Very poor | 2, Poor | 3, Fair | 4, Good | 5, Very good",PSQI component
qol_physical,decimal,Physical QOL Domain,,"0-100, SF-36"
qol_mental,decimal,Mental QOL Domain,,"0-100, SF-36"
"""

print("✓ Patient-Reported Outcomes Dictionary loaded")
print(f"  Variables: {len(pro_data_dictionary.split(chr(10))) - 1}")
print("  Note: Includes PHQ-9, GAD-7, PROMIS, and SF-36 instruments")

✓ Patient-Reported Outcomes Dictionary loaded
  Variables: 15
  Note: Includes PHQ-9, GAD-7, PROMIS, and SF-36 instruments


In [ ]:
# @title
# Example 5: Medical Imaging Data (DICOM-based)
imaging_data_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
accession_number,text,Accession Number,,Unique exam identifier
patient_id,text,Patient ID,,De-identified patient ID
study_date,date,Study Date,,Date imaging was performed
modality,radio,Imaging Modality,"CT, MRI, PET, US, XR",Type of scan
body_region,text,Body Region (RadLex),,"Chest, Abdomen, Brain, etc."
protocol,text,Scan Protocol,,Specific imaging protocol used
slice_thickness,decimal,Slice Thickness (mm),,For CT/MRI
contrast_used,yesno,Contrast Agent Used,"0, No | 1, Yes",IV contrast administration
dose,decimal,Radiation Dose (mGy),,For CT/XR only
finding_present,yesno,Pathologic Finding,"0, No | 1, Yes",Any abnormality detected
finding_type,text,Finding Type,,Description of pathology
lesion_size,decimal,Lesion Size (mm),,Maximum diameter if applicable
radiologist_id,text,Interpreting Radiologist,,Reader ID
dicom_series_uid,text,DICOM Series Instance UID,,Unique series identifier
"""

print("✓ Imaging Data Dictionary loaded")
print(f"  Variables: {len(imaging_data_dictionary.split(chr(10))) - 1}")
print("  Note: Based on DICOM and RadLex standards")

✓ Imaging Data Dictionary loaded
  Variables: 15
  Note: Based on DICOM and RadLex standards


In [ ]:
# @title
# Example 4: Clinical Trial Data (CDISC SDTM-like)
clinical_trial_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
studyid,text,Study Identifier,,Protocol number
usubjid,text,Unique Subject ID,,Unique across all studies
subjid,text,Subject ID,,ID within this study
siteid,text,Site ID,,Clinical site identifier
arm,radio,Treatment Arm,"1, Placebo | 2, Low Dose | 3, High Dose",Randomization arm
visit,text,Visit Name,,"Screening, Baseline, Week 4, Week 8, Week 12"
visitnum,integer,Visit Number,,"1-10"
visitdate,date,Visit Date,,Actual visit date
ae_term,text,Adverse Event Term,,MedDRA preferred term
ae_severity,radio,AE Severity,"1, Mild | 2, Moderate | 3, Severe",Intensity grading
ae_serious,yesno,Serious AE,"0, No | 1, Yes",SAE flag
ae_related,radio,Related to Study Drug,"0, Unrelated | 1, Unlikely | 2, Possible | 3, Probable | 4, Definite",Causality assessment
efficacy_score,decimal,Primary Efficacy Score,,"0-100, higher is better"
qol_score,decimal,Quality of Life Score,,"0-100, SF-36"
compliance,decimal,Medication Compliance (%),,"0-100, pill count"
"""

print("✓ Clinical Trial Data Dictionary loaded")
print(f"  Variables: {len(clinical_trial_dictionary.split(chr(10))) - 1}")
print("  Note: Follows CDISC SDTM standards")

✓ Clinical Trial Data Dictionary loaded
  Variables: 16
  Note: Follows CDISC SDTM standards


In [ ]:
# @title
# Example 3: Genomic/Genetic Data
genomic_data_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
sample_id,text,Sample ID,,Unique biospecimen identifier
patient_id,text,Patient ID,,De-identified patient identifier
gene_symbol,text,Gene Symbol (HGNC),,Official gene symbol from HGNC
variant_id,text,Variant ID (dbSNP),,rs number from dbSNP database
chromosome,text,Chromosome,,"1-22, X, Y, MT"
position,integer,Genomic Position (hg38),,Position on reference genome GRCh38
ref_allele,text,Reference Allele,,"A, C, G, T"
alt_allele,text,Alternate Allele,,"A, C, G, T, or indel"
variant_type,radio,Variant Type,"1, SNV | 2, Insertion | 3, Deletion | 4, CNV",Single nucleotide or structural
genotype,text,Genotype,,"0/0, 0/1, 1/1"
read_depth,integer,Read Depth (DP),,Number of reads covering position
allele_frequency,decimal,Allele Frequency (AF),,"0.0-1.0, population frequency"
clinical_significance,radio,Clinical Significance,"0, Benign | 1, Likely Benign | 2, VUS | 3, Likely Pathogenic | 4, Pathogenic",ClinVar classification
phenotype_association,text,Associated Phenotype,,Disease or trait association
transcript_id,text,Transcript ID (Ensembl),,Canonical transcript identifier
"""

print("✓ Genomic Data Dictionary loaded")
print(f"  Variables: {len(genomic_data_dictionary.split(chr(10))) - 1}")
print("  Note: Follows HGNC, dbSNP, and ClinVar standards")

✓ Genomic Data Dictionary loaded
  Variables: 16
  Note: Follows HGNC, dbSNP, and ClinVar standards


In [ ]:
# @title
# Example 2: OMOP Common Data Model (CDM) Format
omop_data_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
person_id,integer,Person ID,,OMOP person identifier
visit_occurrence_id,integer,Visit Occurrence ID,,Foreign key to VISIT_OCCURRENCE
measurement_date,date,Measurement Date,,Date of measurement
measurement_concept_id,integer,Measurement Concept ID,,OMOP standard concept for measurement type
value_as_number,decimal,Numeric Value,,Numeric result value
value_as_concept_id,integer,Categorical Value Concept ID,,OMOP concept for categorical results
unit_concept_id,integer,Unit Concept ID,,OMOP concept for unit of measure
range_low,decimal,Normal Range Lower Bound,,Lower limit of normal range
range_high,decimal,Normal Range Upper Bound,,Upper limit of normal range
provider_id,integer,Provider ID,,Foreign key to PROVIDER table
measurement_source_value,text,Source Value,,Original value from source system
measurement_source_concept_id,integer,Source Concept ID,,Concept ID from source vocabulary
"""

print("✓ OMOP CDM Data Dictionary loaded")
print(f"  Variables: {len(omop_data_dictionary.split(chr(10))) - 1}")
print("  Note: Uses OMOP standard concept IDs for interoperability")

✓ OMOP CDM Data Dictionary loaded
  Variables: 13
  Note: Uses OMOP standard concept IDs for interoperability


In [ ]:
# @title
# Example 1: Electronic Health Record (EHR) Data
ehr_data_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
mrn,text,Medical Record Number,,Unique patient identifier
encounter_id,text,Encounter ID,,Unique visit identifier
visit_date,date,Visit Date,,Date of clinical encounter
chief_complaint,text,Chief Complaint,,Primary reason for visit
dx_code,text,Diagnosis Code (ICD-10),,Primary diagnosis
bp_systolic,integer,Systolic BP (mmHg),,"70-250, sitting position"
bp_diastolic,integer,Diastolic BP (mmHg),,"40-150, sitting position"
heart_rate,integer,Heart Rate (bpm),,"40-200"
temperature,decimal,Temperature (F),,"95.0-106.0"
respiratory_rate,integer,Respiratory Rate (breaths/min),,"8-40"
oxygen_sat,integer,Oxygen Saturation (%),,"70-100, room air"
bmi,decimal,Body Mass Index,,Calculated from height/weight
smoking_status,radio,Smoking Status,"0, Never | 1, Former | 2, Current",From social history
medication_count,integer,Number of Active Medications,,Count of current prescriptions
lab_ordered,yesno,Labs Ordered,"0, No | 1, Yes",Any lab tests ordered this visit
"""

print("✓ EHR Data Dictionary loaded")
print(f"  Variables: {len(ehr_data_dictionary.split(chr(10))) - 1}")

✓ EHR Data Dictionary loaded
  Variables: 16


### 📚 Example Data Dictionaries by Healthcare Domain

Below are real-world examples for different healthcare data types. Use these to test the system or as templates for your own data.

## 7. Example Usage and Demonstration

Let's demonstrate the system with a sample healthcare data dictionary.

In [ ]:
# @title
# Sample REDCap-style data dictionary
sample_data_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
patient_id,text,Patient ID,,Unique identifier
age,integer,Age (years),,Age at enrollment
sex,radio,Biological Sex,"1, Male | 2, Female | 3, Other",
bp_systolic,integer,Systolic Blood Pressure (mmHg),,
bp_diastolic,integer,Diastolic Blood Pressure (mmHg),,
diagnosis_date,date,Diagnosis Date,,Date of primary diagnosis
hba1c,decimal,Hemoglobin A1c (%),,Glycated hemoglobin
"""

In [ ]:
# @title
# Initialize orchestrator
orchestrator = Orchestrator(db)

# Create some useful Toons for context
print("Creating Toons for context management...")

# Helper function to create or update a Toon
def create_or_update_toon(name: str, toon_type: ToonType, content: str, metadata: Optional[Dict] = None):
    existing_toon = orchestrator.toon_manager.get_toon_by_name(name)
    if existing_toon:
        orchestrator.toon_manager.update_toon(existing_toon.toon_id, content=content, metadata=metadata)
        print(f"Updated existing Toon '{name}' (ID: {existing_toon.toon_id})")
    else:
        orchestrator.toon_manager.create_toon(name, toon_type, content, metadata)

# Instruction Toon for OMOP mapping
create_or_update_toon(
    name="OMOP_Mapping_Instructions",
    toon_type=ToonType.INSTRUCTION,
    content="""When mapping to OMOP CDM:
- Blood pressure measurements should map to OMOP concept_id 3004249 (Systolic) and 3012888 (Diastolic)
- HbA1c should map to OMOP concept_id 3004410
- Age should be stored as an integer in years
- Sex should use standard OMOP gender concepts: 8507 (Male), 8532 (Female)
"""
)

# Design decision Toon
create_or_update_toon(
    name="Project_Design_Notes",
    toon_type=ToonType.DESIGN,
    content="""This is a diabetes research study collecting baseline clinical measurements.
All measurements follow standard clinical protocols. Blood pressure is measured in sitting position
after 5 minutes rest. HbA1c measured using DCCT-aligned assay.
"""
)

print("\nToons created/updated successfully!")

✓ Orchestrator initialized with 10 req/min limit
Creating Toons for context management...
Updated Toon ID 1
Updated existing Toon 'OMOP_Mapping_Instructions' (ID: 1)
Updated Toon ID 2
Updated existing Toon 'Project_Design_Notes' (ID: 2)

Toons created/updated successfully!


In [ ]:
# @title
# Inject Toons into relevant agents
toons = orchestrator.toon_manager.list_toons()
orchestrator.domain_ontology.inject_toons(toons)
orchestrator.plain_language.inject_toons(toons)

print(f"Injected {len(toons)} Toons into agent context")

Injected 2 Toons into agent context


In [ ]:
# @title
# Process the data dictionary
# NOTE: Set AUTO_APPROVE_MODE = True only for testing/demo purposes
# For normal usage, keep it False to enable manual review workflow
AUTO_APPROVE_MODE = False # Change this to True to skip manual review
job_id = orchestrator.process_data_dictionary(
    source_data=sample_data_dictionary,
    source_file="diabetes_study_data_dictionary.csv",
    auto_approve=AUTO_APPROVE_MODE
)
# Provide clear feedback about the mode
print(f"\n{'='*60}")
print(f"Job ID: {job_id}")
print(f"Auto-approve mode: {'ENABLED' if AUTO_APPROVE_MODE else 'DISABLED'}")
print(f"{'='*60}")
if AUTO_APPROVE_MODE:
  print("\n⚠️  AUTO-APPROVE is ENABLED")
  print("   → All items were automatically approved")
  print("   → Skip to Cell 48 to generate final documentation")
  print("   → Items are in the APPROVED queue, not PENDING")
else:
  print("\n✓ Manual review mode is ACTIVE")
  print("   → Items are awaiting review in PENDING queue")
  print("   → Run Cell 44 to see pending items")
  print("   → Use Cells 45-47 to review and approve")
  print(f"{'='*60}\n")

Created job 7b8e60a5eacc for source file: diabetes_study_data_dictionary.csv

=== Step 1: Parsing Data ===
Parsed 7 variables

=== Step 2: Technical Analysis ===
Analyzed 7 variables

=== Step 3: Domain Ontology Mapping ===
Mapping variable 1/7: patient_id
Mapping variable 2/7: age
⏱️  Rate limiting: waiting 5.0s...
Mapping variable 3/7: sex
⏱️  Rate limiting: waiting 5.1s...
Mapping variable 4/7: systolic_blood_pressure
⏱️  Rate limiting: waiting 4.9s...
Mapping variable 5/7: diastolic_blood_pressure
⏱️  Rate limiting: waiting 4.9s...
Mapping variable 6/7: diagnosis_date
⏱️  Rate limiting: waiting 5.1s...
Mapping variable 7/7: hba1c
⏱️  Rate limiting: waiting 5.1s...

Enriched 7 variables with ontology mappings

=== Step 4: Plain Language Documentation ===
Documenting variable 1/7: patient_id
Documenting variable 2/7: age
⏱️  Rate limiting: waiting 4.7s...
Documenting variable 3/7: sex
⏱️  Rate limiting: waiting 4.7s...


⚠️  Rate limit hit. Waiting 6.0s before retry 1/3...
Documenting variable 4/7: systolic_blood_pressure
⏱️  Rate limiting: waiting 4.9s...
Documenting variable 5/7: diastolic_blood_pressure
⏱️  Rate limiting: waiting 4.3s...
Documenting variable 6/7: diagnosis_date
⏱️  Rate limiting: waiting 4.1s...
Documenting variable 7/7: hba1c
⏱️  Rate limiting: waiting 4.8s...

✓ Job 7b8e60a5eacc processing complete
  - 7 items ready for review

Job ID: 7b8e60a5eacc
Auto-approve mode: DISABLED

✓ Manual review mode is ACTIVE
   → Items are awaiting review in PENDING queue
   → Run Cell 44 to see pending items
   → Use Cells 45-47 to review and approve



In [ ]:
# @title
# Review pending items (requires auto_approve=False in previous cell)
# This cell validates that job_id exists and provides detailed status info
# Validate job_id exists
if 'job_id' not in locals() and 'job_id' not in globals():
  print("⚠️ ERROR: job_id variable not found!")
  print("   Please run Cell 43 first to create a job.")
  pending_items = []
else:
  print(f"Checking review queue for job: {job_id}\n")
  # Get pending items
  pending_items = orchestrator.review_queue.get_pending_items(job_id)
  # Also check total items and status distribution
  all_items_result = db.execute_query(
        "SELECT COUNT(*) as count FROM ReviewQueue WHERE job_id = ?",
        (job_id,)
    )
  total_items = all_items_result[0]['count'] if all_items_result else 0

  status_breakdown = []
  if total_items > 0:
    status_breakdown = db.execute_query("""
        SELECT status, COUNT(*) as count
        FROM ReviewQueue
        WHERE job_id = ?
        GROUP BY status
    """, (job_id,))
  # Display results
  print(f"{'='*60}")
  print(f"Review Queue Status for Job: {job_id}")
  print(f"{'='*60}")
  print(f"Total items in job: {total_items}")
  print(f"Pending review items: {len(pending_items)}\n")
  if status_breakdown:
    print("Status breakdown:")
    for row in status_breakdown:
      print(f"  {row['status']}: {row['count']} items")
      print(f"{'='*60}\n")
      # Provide guidance based on what we found
      if total_items == 0:
        print("⚠️ No items found for this job!")
        print("   The job may have failed or no items were created.")
      elif len(pending_items) == 0 and total_items > 0:
        print("ℹ️  All items have been processed (no pending reviews)")
        print("   This happens when:")
        print("   • auto_approve=True was used (items auto-approved)")
        print("   • Items were manually approved/rejected")
        print("\n   To review approved items, run:")
        print("   approved_items = orchestrator.review_queue.get_approved_items(job_id)")
      elif pending_items:
        print(f"✓ Found {len(pending_items)} items awaiting review")
        print("   Starting interactive review process...")

        # Interactive review loop
        for item in pending_items:
            print(f"\n{'='*80}")
            print(f"REVIEW ITEM {item.item_id} (Source: {item.source_agent})")
            print(f"{'='*80}")
            print("\n--- GENERATED CONTENT ---\n")
            print(item.generated_content)
            print("\n---------------------------\n")

            action = input("Action (a: approve, e: edit & approve, r: reject, s: submit clarification, skip: skip item): ").lower().strip()

            if action == 'a':
                orchestrator.review_queue.approve_item(item.item_id)
                print(f"Item {item.item_id} approved.")
            elif action == 'e':
                edited_content = input("Enter edited content (leave blank to use original): ")
                if not edited_content.strip():
                    edited_content = item.generated_content
                orchestrator.review_queue.approve_item(item.item_id, edited_content)
                print(f"Item {item.item_id} approved with edits.")
            elif action == 'r':
                feedback = input("Enter rejection feedback: ")
                orchestrator.review_queue.reject_item(item.item_id, feedback)
                print(f"Item {item.item_id} rejected.")
            elif action == 's':
                clarification_response = input("Enter clarification response: ")
                orchestrator.review_queue.submit_clarification(item.item_id, clarification_response)
                print(f"Item {item.item_id} submitted for clarification.")
            elif action == 'skip':
                print(f"Item {item.item_id} skipped.")
            else:
                print("Invalid action. Item skipped.")
            print("\n")
        print("Interactive review process completed.")

Checking review queue for job: 7b8e60a5eacc

Review Queue Status for Job: 7b8e60a5eacc
Total items in job: 7
Pending review items: 6

Status breakdown:
  Approved: 1 items

✓ Found 6 items awaiting review
   Starting interactive review process...

REVIEW ITEM 23 (Source: PlainLanguageAgent)

--- GENERATED CONTENT ---

## Variable: age

**Description:** Age of the participant at the time of enrollment in the study.

**Technical Details:**
- Data Type: Integer (years)
- Cardinality: Required
- Valid Values: Non-negative integer representing age in years.

**Standard Ontology Mappings:**
- OMOP: Age in Years - Age in Years

**Clinical Context:** Age is a fundamental demographic variable and an important risk factor for diabetes and related complications.  It is essential for risk stratification, cohort characterization, and adjusting for age-related effects in statistical analyses.

---------------------------

Action (a: approve, e: edit & approve, r: reject, s: submit clarification, ski

In [ ]:
# @title
# Finalize and generate documentation
final_documentation = orchestrator.finalize_documentation(
    job_id=job_id,
    output_file="healthcare_data_documentation.md"
)

print("\n=== Final Documentation Preview ===")
print(final_documentation[:1000] + "\n...")


=== Assembling Final Documentation ===
✓ Documentation saved to healthcare_data_documentation.md

=== Final Documentation Preview ===
# Healthcare Data Documentation

**Generated:** 2025-11-16 00:40:05
**Job ID:** 7b8e60a5eacc

---

## Table of Contents

1. [patient_id](#patient_id)
2. [age](#age)
3. [sex](#sex)
4. [systolic_blood_pressure](#systolic_blood_pressure)
5. [diastolic_blood_pressure](#diastolic_blood_pressure)
6. [diagnosis_date](#diagnosis_date)
7. [hba1c](#hba1c)

---

## Variable: patient_id

**Description:** Unique identifier for each patient in the study.

**Technical Details:**
- Data Type: String
- Cardinality: Required
- Valid Values: Alphanumeric string, unique within the dataset

**Standard Ontology Mappings:**
- OMOP: 1145515 - Person Source Value

**Clinical Context: [EDITED BY REVIEWER]** This variable is essential for linking patient data across different tables and for tracking individuals throughout the study. It is critical for maintaining patient confiden

## 8. Context Management - Working vs Long-Term Memory

Demonstration of the memory model and context compaction.

In [ ]:
# @title
class ContextManager:
    """Manages working and long-term memory for the ADE system."""

    def __init__(self, db_manager: DatabaseManager, max_tokens: int = 100000):
        self.db = db_manager
        self.max_tokens = max_tokens
        self.compaction_threshold = int(max_tokens * 0.8)

    def estimate_tokens(self, text: str) -> int:
        """Rough token estimation (1 token ≈ 4 characters)."""
        return len(text) // 4

    def get_working_memory(self, job_id: str) -> Dict[str, Any]:
        """Get current working memory for a job."""
        # Get active toons
        query = "SELECT state_value FROM SystemState WHERE state_key = ?"
        result = self.db.execute_query(query, (f"active_toons_{job_id}",))
        active_toon_ids = json.loads(result[0]['state_value']) if result else []

        # Get session history
        query = "SELECT * FROM SessionHistory WHERE job_id = ? ORDER BY created_at"
        history_rows = self.db.execute_query(query, (job_id,))

        session_history = []
        for row in history_rows:
            session_history.append({
                'role': row['role'],
                'content': row['content'],
                'timestamp': row['created_at']
            })

        # Calculate total tokens
        total_tokens = sum(self.estimate_tokens(msg['content']) for msg in session_history)

        return {
            'active_toon_ids': active_toon_ids,
            'session_history': session_history,
            'total_tokens': total_tokens,
            'needs_compaction': total_tokens > self.compaction_threshold
        }

    def add_to_session_history(self, job_id: str, role: str, content: str):
        """Add a message to session history."""
        query = """
        INSERT INTO SessionHistory (job_id, role, content)
        VALUES (?, ?, ?)
        """
        self.db.execute_update(query, (job_id, role, content))

    def compact_context(self, job_id: str) -> str:
        """Compact session history using summarization."""
        print("\n=== Context Compaction Triggered ===")

        working_memory = self.get_working_memory(job_id)
        session_history = working_memory['session_history']

        # Build conversation text
        conversation = "\n\n".join([
            f"{msg['role'].upper()}: {msg['content']}"
            for msg in session_history
        ])

        # Use Gemini to summarize
        compactor_prompt = f"""
You are a ContextCompactorAgent. Your task is to create a concise summary of this
conversation that preserves all critical information, decisions, and clarifications.

Conversation:
{conversation}

Provide a structured summary that includes:
1. Key decisions made
2. Important clarifications provided
3. Current state of the work

Summary:
"""

        model = genai.GenerativeModel("gemini-2.0-flash-exp")
        response = model.generate_content(compactor_prompt)
        summary = response.text

        print(f"Original tokens: {working_memory['total_tokens']}")
        print(f"Summary tokens: {self.estimate_tokens(summary)}")
        print(f"Reduction: {100 * (1 - self.estimate_tokens(summary) / working_memory['total_tokens']):.1f}%")

        # Store summary as a Toon
        toon_manager = ToonManager(self.db)
        toon_manager.create_toon(
            name=f"Session_Summary_{job_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            toon_type=ToonType.SUMMARY,
            content=summary,
            metadata={'job_id': job_id, 'original_tokens': working_memory['total_tokens']}
        )

        return summary

    def clear_context(self, job_id: str):
        """Clear working memory (flush session history and active toons)."""
        # Note: This doesn't delete from database, just resets the "active" state
        query = "UPDATE SystemState SET state_value = '[]' WHERE state_key = ?"
        self.db.execute_update(query, (f"active_toons_{job_id}",))

        print(f"Cleared working memory for job {job_id}")
        print("Note: Session history preserved in long-term memory (database)")

In [ ]:
# @title
# Demonstrate context management
context_manager = ContextManager(db)

# Simulate a conversation
context_manager.add_to_session_history(job_id, "user", "Please process the diabetes data dictionary")
context_manager.add_to_session_history(job_id, "assistant", "I'll process the data dictionary through the agent pipeline.")
context_manager.add_to_session_history(
    job_id,
    "user",
    "For the HbA1c variable, please note that values above 6.5% indicate diabetes diagnosis."
)

# Check working memory
working_memory = context_manager.get_working_memory(job_id)
print(f"\nWorking Memory Status:")
print(f"  Total tokens: {working_memory['total_tokens']}")
print(f"  Needs compaction: {working_memory['needs_compaction']}")
print(f"  Session messages: {len(working_memory['session_history'])}")


Working Memory Status:
  Total tokens: 92
  Needs compaction: False
  Session messages: 6


## 9. System Status and Monitoring

In [ ]:
# @title
def display_system_status(db: DatabaseManager):
    """Display current system status."""
    print("\n" + "="*80)
    print("ADE SYSTEM STATUS")
    print("="*80)

    # Jobs
    jobs = db.execute_query("SELECT * FROM Jobs")
    print(f"\nJobs: {len(jobs)}")
    for job in jobs:
        print(f"  [{job['job_id']}] {job['source_file']} - Status: {job['status']}")

    # Toons
    toons = db.execute_query("SELECT toon_type, COUNT(*) as count FROM Toons GROUP BY toon_type")
    print(f"\nToon Library:")
    for toon in toons:
        print(f"  {toon['toon_type']}: {toon['count']}")

    # Review Queue
    review_stats = db.execute_query(
        "SELECT status, COUNT(*) as count FROM ReviewQueue GROUP BY status"
    )
    print(f"\nReview Queue:")
    for stat in review_stats:
        print(f"  {stat['status']}: {stat['count']}")

    print("\n" + "="*80)

display_system_status(db)


ADE SYSTEM STATUS

Jobs: 4
  [ebb125fa559e] diabetes_study_data_dictionary.csv - Status: Running
  [37f922d3a1cd] diabetes_study_data_dictionary.csv - Status: Completed
  [a82e335d1a8e] diabetes_study_data_dictionary.csv - Status: Running
  [7b8e60a5eacc] diabetes_study_data_dictionary.csv - Status: Completed

Toon Library:
  Toon_Design: 1
  Toon_Instruction: 1

Review Queue:
  Approved: 7
  Pending: 21



## 10. Export and Cleanup

In [ ]:
# @title
# Export database for backup
import shutil

def backup_database(db_path: str, backup_path: str):
    """Create a backup of the project database."""
    shutil.copy2(db_path, backup_path)
    print(f"Database backed up to {backup_path}")

# Uncomment to create backup
backup_database("project.db", "project_backup.db")

Database backed up to project_backup.db


In [ ]:
# @title
# View generated documentation
if os.path.exists("healthcare_data_documentation.md"):
    with open("healthcare_data_documentation.md", 'r') as f:
        print("\n=== Generated Documentation ===")
        print(f.read())


=== Generated Documentation ===
# Healthcare Data Documentation

**Generated:** 2025-11-16 00:40:05
**Job ID:** 7b8e60a5eacc

---

## Table of Contents

1. [patient_id](#patient_id)
2. [age](#age)
3. [sex](#sex)
4. [systolic_blood_pressure](#systolic_blood_pressure)
5. [diastolic_blood_pressure](#diastolic_blood_pressure)
6. [diagnosis_date](#diagnosis_date)
7. [hba1c](#hba1c)

---

## Variable: patient_id

**Description:** Unique identifier for each patient in the study.

**Technical Details:**
- Data Type: String
- Cardinality: Required
- Valid Values: Alphanumeric string, unique within the dataset

**Standard Ontology Mappings:**
- OMOP: 1145515 - Person Source Value

**Clinical Context: [EDITED BY REVIEWER]** This variable is essential for linking patient data across different tables and for tracking individuals throughout the study. It is critical for maintaining patient confidentiality by ensuring that the actual patient identifiers are not directly stored or used in analyses. T

## Summary

This notebook demonstrates a complete implementation of the Agent Development Environment (ADE) for Healthcare Data Documentation. Key features:

### Implemented Components:
1. **SQLite Database** - Full schema with all required tables
2. **Toon System** - Context management with 6 Toon types
3. **Review Queue** - Complete HITL workflow with status management
4. **Core Agents**:
   - DataParserAgent
   - TechnicalAnalyzerAgent
   - DomainOntologyAgent
   - PlainLanguageAgent
   - DocumentationAssemblerAgent
5. **Orchestrator** - Agent workflow management
6. **Context Manager** - Working vs long-term memory with compaction

### Key Workflows:
- Data ingestion and parsing
- Technical analysis with clarification requests
- Ontology mapping (OMOP, LOINC, SNOMED)
- Human-readable documentation generation
- Human-in-the-loop review and approval
- Context compaction for large sessions

### Next Steps:
To use this in Kaggle:
1. Add your Google Gemini API key as a Kaggle secret named 'GOOGLE_API_KEY'
2. Upload your data dictionary files
3. Run the notebook cells in order
4. Review and approve generated documentation
5. Export the final documentation

### Extending the System:
- Add custom agents for domain-specific processing
- Create new Toon types for your use case
- Implement additional ontology mappings
- Build a web UI using Streamlit or Gradio
- Add version control for documentation